# Example jupyternotebook workflow for pyjugex

This notebook demonstrates how a gene differential analysis can be carried out with pyjugex. Also demonstrated is the use of atlas-core to retrieve probability maps.

## Download probability maps via atlas core (optional)

You may choose to use `ebrains_atlascore` to download probabilistic maps. Alternatively, you can use a variety of tools, such as `wget` on command line or `requests` in python to download the necessary probabilistic maps.

It should be noted that Allen Brain provides data in MNI152 reference space. In normal circumstances, you should probably also use probabilistic maps in that space.

In [ ]:
!pip install ebrains_atlascore

In [ ]:
from ebrains_atlascore import regions
from ebrains_atlascore.util.hemisphere import Hemisphere
from ebrains_atlascore.region import Region
pmap_hoc1 = regions.get_probability_map_for_region(Region('Area-hOc1', parcellation='JuBrain Cytoarchitectonic Atlas', referencespace='MNI152'), Hemisphere.LEFT.value, 0.2)
pmap_hoc2 = regions.get_probability_map_for_region(Region('Area-hOc2', parcellation='JuBrain Cytoarchitectonic Atlas', referencespace='MNI152'), Hemisphere.LEFT.value, 0.2)


As `nibabel` does not load file in memory, write them to disk.

In [ ]:
with open('hoc1.nii', 'wb') as fp:
    fp.write(pmap_hoc1.data)
with open('hoc2.nii', 'wb') as fp:
    fp.write(pmap_hoc2.data)

## pyjugex analysis

This section details how one may set up parameters for gene differential analysis.

In [ ]:
# install pyjugex and import dependencies

!pip install pyjugex
import pyjugex
import nibabel as nib

In [ ]:
# setup parameters

gene_list=['MAOA','TAC1']
nii1 = nib.load('hoc1.nii')
nii2 = nib.load('hoc2.nii')

In [ ]:
# load parameters and setup analysis

analysis = pyjugex.analysis(
  n_rep=1000,
  gene_list=gene_list,
  roi1 = nii1,
  roi2 = nii2
)

In [ ]:
# prior to analysis, one can retrieve the coordinates of the probes in MNI152 space

filtered_coord = analysis.get_filtered_coord()
assert(len(filtered_coord['roi1']) == 12)
assert(len(filtered_coord['roi2']) == 11)

In [ ]:
analysis.run() # Go grab a coffee

In [ ]:
# results of the differential analysis is saved in the result object

maoa = analysis.anova.result.get('MAOA')
tac1 = analysis.anova.result.get('TAC1')

assert(0.95 <= maoa <= 1.0)
assert(0.35 <= tac1 <= 0.55)

In [ ]:
# alter the parameter and start another run

analysis.n_rep = 10000
analysis.run() # Really go grab a coffee

In [ ]:
maoa = analysis.anova.result.get('MAOA')
tac1 = analysis.anova.result.get('TAC1')

assert(0.95 <= maoa <= 1.0)
assert(0.35 <= tac1 <= 0.52)